In [2]:
import h5py
import numpy as np
import pandas as pd
from scipy.signal import spectrogram
import librosa
from tqdm import tqdm

In [ ]:
# Parameters for spectrogram
FS = 100
NPERSEG = 64
NOVERLAP = 48

def compute_spectrograms(waveforms, fs=FS, nperseg=NPERSEG, noverlap=NOVERLAP):
    """
    Compute three-channel spectrogram and log-normalized spectrogram.
    """
    specs = []
    for i in range(3):
        f, t, Sxx = spectrogram(waveforms[i], fs=fs, nperseg=nperseg, noverlap=noverlap)
        specs.append(Sxx)
    specs = np.stack(specs, axis=0)  # (3, F, T)
    # Log normalization
    log_specs = np.log1p(specs)
    return specs, log_specs

def extract_features(waveforms, fs=FS):
    """
    Extract additional features to help classify pre- vs post-events.

    Features:
    - Max and RMS amplitude per channel
    - Spectral centroid and bandwidth (mean over time)
    - Zero-crossing rate (mean)
    - Energy ratio in a specific frequency band (e.g., 5–10 Hz)
    """
    features = {}

    # Basic amplitude features
    for i in range(3):
        ch_data = waveforms[i]
        features[f'max_amp_ch{i}'] = np.max(np.abs(ch_data))
        features[f'rms_amp_ch{i}'] = np.sqrt(np.mean(ch_data**2))

    # Advanced spectral and temporal features using librosa
    # For each channel, compute spectral centroid, bandwidth, and zero-crossing rate
    for i in range(3):
        ch_data = waveforms[i]

        # Spectral centroid & bandwidth
        centroid = librosa.feature.spectral_centroid(y=ch_data, sr=fs)
        bandwidth = librosa.feature.spectral_bandwidth(y=ch_data, sr=fs)
        # Take mean over time frames
        features[f'spectral_centroid_ch{i}'] = np.mean(centroid)
        features[f'spectral_bandwidth_ch{i}'] = np.mean(bandwidth)

        # Zero-crossing rate
        zcr = librosa.feature.zero_crossing_rate(y=ch_data)
        features[f'zcr_ch{i}'] = np.mean(zcr)

        # Energy ratio in a band (e.g., 5–10 Hz)
        # Compute FFT
        fft_vals = np.fft.rfft(ch_data)
        freqs = np.fft.rfftfreq(len(ch_data), 1/fs)
        
        # Total energy
        total_energy = np.sum(np.abs(fft_vals)**2)
        
        # Frequency band energy: 5–10 Hz
        band_mask = (freqs >= 5) & (freqs <= 10)
        band_energy = np.sum(np.abs(fft_vals[band_mask])**2)
        
        # Ratio of band energy to total energy
        band_ratio = band_energy / (total_energy + 1e-12)  # Avoid division by zero
        features[f'band_5_10_energy_ratio_ch{i}'] = band_ratio

    return features

# Files
folders = ['MC2', 'MMO1', 'NRCA', 'T1212', 'T1213', 'T1216', 'T1214', 'T1244']
data_rows = []
for folder in folders:
    print(f"Processing folder {folder}")
    pre_file = fr"C:\Users\frmar\OneDrive\Desktop\EQML Project\data\dataset\{folder}\{folder}_waveforms_pre.hdf5"
    post_file = fr"C:\Users\frmar\OneDrive\Desktop\EQML Project\data\dataset\{folder}\{folder}_waveforms_post.hdf5"
    # Process pre file
    with h5py.File(pre_file, 'r') as hdf:
        for event_id in tqdm(hdf.keys()):
            waveforms = hdf[event_id][()]  # (3, 2500)
            try:
                # Compute spectrograms
                specs, log_specs = compute_spectrograms(waveforms)
                
                # Extract features
                feat = extract_features(waveforms)
                
                # Create row dict
                row = {
                    'ID': event_id,
                    'label': 'pre',
                    'waveform': waveforms,
                    'spectrograms': specs,        # (3, F, T)
                    'log_spectrograms': log_specs # (3, F, T)
                }
                # Add extracted features to row
                row.update(feat)
                
                data_rows.append(row)

            except Exception as e:
                print(f"Error {e} processing event {event_id}")

    # Process post file
    with h5py.File(post_file, 'r') as hdf:
        for event_id in tqdm(hdf.keys()):
            try:
                waveforms = hdf[event_id][()]  # (3, 2500)
                
                # Compute spectrograms
                specs, log_specs = compute_spectrograms(waveforms)
                
                # Extract features
                feat = extract_features(waveforms)
                
                # Create row dict
                row = {
                    'ID': event_id,
                    'label': 'post',
                    'waveform': waveforms,
                    'spectrograms': specs,
                    'log_spectrograms': log_specs
                }
                row.update(feat)
                
                data_rows.append(row)
            except Exception as e:
                print(f"Error {e} processing event {event_id}")
    # Create DataFrame
    df = pd.DataFrame(data_rows)
    print('Saving DataFrame')
    df.to_pickle(f"data/{folder}_dataframe.pkl")
    print('DataFrame saved')

Processing folder MC2


100%|██████████| 2804/2804 [00:18<00:00, 148.10it/s]


Saving DataFrame
